In [1]:
import pandas as pd
import numpy as np
from langdetect import detect
import re 
import nltk
from nltk.corpus import stopwords
import unicodedata
from rapidfuzz import process, fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import requests
import json
import fitz
from tqdm import tqdm

In [2]:
# Importing data 
cards = pd.read_csv('cards_trello.csv')
cards.head(2)

,id,dateLastActivity,desc,due,name,label_names
0,684c1005b986bba107a1917a,2025-06-13T11:48:36.998Z,about the job job description o time de analy...,2025-07-25T11:44:00.000Z,Cientista de Dados Pleno - Porto - LinkedIn,['Enviado']
1,684c138a5be4f9abe78529e7,2025-06-13T12:03:36.307Z,about the job deseja construir um futuro sóli...,2025-07-25T11:44:00.000Z,Analist ade Dados - Cargill - LinkedIn,['Enviado']


Preprocessing data 

In [3]:
cards['desc_lang'] = [detect(str(x)) for x in cards['desc']]

In [4]:
cards.head(2)

,id,dateLastActivity,desc,due,name,label_names,desc_lang
0,684c1005b986bba107a1917a,2025-06-13T11:48:36.998Z,about the job job description o time de analy...,2025-07-25T11:44:00.000Z,Cientista de Dados Pleno - Porto - LinkedIn,['Enviado'],pt
1,684c138a5be4f9abe78529e7,2025-06-13T12:03:36.307Z,about the job deseja construir um futuro sóli...,2025-07-25T11:44:00.000Z,Analist ade Dados - Cargill - LinkedIn,['Enviado'],pt


In [5]:
cards['desc_lang'].value_counts()

desc_lang
pt    225
en     93
tl      4
es      1
Name: count, dtype: int64

In [6]:
replace_values = {'pt' : 'portuguese', 'en' : 'english', 'tl' : 'portuguese', 'es': 'spanish'}

cards['desc_lang'] = cards['desc_lang'].replace(replace_values)

In [7]:
def substituir_sinonimos(texto):
    texto = texto.lower()
    texto = texto.replace('business intelligence', 'bi')
    texto = texto.replace('inteligência de negócios', 'bi')
    return texto

def cleaning_soft(text):

    text = str(text)
    text = text.lower()
    text = re.sub(r'[^\w\s-]', '', text)
    text = unicodedata.normalize('NFD', text)
    text = re.sub(r'[\u0300-\u036f]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = substituir_sinonimos(text)

    return text.strip()



def cleaning_text(text, lang):
    stopwords_lang = set(stopwords.words(lang))

    text = cleaning_soft(text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub('about the job', '', text)
    text = re.sub('job description', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    words = [p for p in words if p not in stopwords_lang]

    return ' '.join(words)



In [8]:
cards['desc_clean'] = [cleaning_text(text, lang) for text, lang in zip(cards['desc'], cards['desc_lang'])]

In [9]:
cards['name_clean'] = [cleaning_soft(text) for text in cards['name']]


In [10]:
cards[['vaga', 'empresa', 'fonte']] = cards['name_clean'].str.split('-', n = 2, expand= True)
cards['vaga'] = [re.sub(r'\b(junior|pleno|pl|senior|jr)\b', '', text, flags=re.IGNORECASE) for text in cards['vaga']]
cards['vaga'] = cards['vaga'].apply(cleaning_soft)
cards.head(2)

,id,dateLastActivity,desc,due,name,label_names,desc_lang,desc_clean,name_clean,vaga,empresa,fonte
0,684c1005b986bba107a1917a,2025-06-13T11:48:36.998Z,about the job job description o time de analy...,2025-07-25T11:44:00.000Z,Cientista de Dados Pleno - Porto - LinkedIn,['Enviado'],portuguese,time analytics insights busca profissional hab...,cientista de dados pleno - porto - linkedin,cientista de dados,porto,linkedin
1,684c138a5be4f9abe78529e7,2025-06-13T12:03:36.307Z,about the job deseja construir um futuro sóli...,2025-07-25T11:44:00.000Z,Analist ade Dados - Cargill - LinkedIn,['Enviado'],portuguese,deseja construir futuro solido sustentavel des...,analist ade dados - cargill - linkedin,analist ade dados,cargill,linkedin


In [11]:
vagas_padrao = ['analista de dados', 'cientista de dados', 'analista de bi', 'data analyst',
                'data scientist', 'bi analyst', 'product analyst', 'analista atuarial', 'estatistico', 'especialista bi']

replace_names = {
    'data scientist': 'cientista de dados',
    'data analyst': 'analista de dados',
    'bi analyst': 'analista de bi',
    'especialista bi': 'analista de bi',
    'trainee engenheiro de dta analytics ai': 'engenheiro de dados'
}

In [12]:

# Fallback: verifica se todos os tokens da vaga padrão estão presentes na vaga suja
def fallback_match(vaga_suja, lista_padrao):
    vaga_tokens = set(vaga_suja.lower().split())
    for vaga_padrao in lista_padrao:
        tokens_padrao = set(vaga_padrao.lower().split())
        if tokens_padrao.issubset(vaga_tokens):
            return vaga_padrao
    return None

# Função principal
def padronizar_vaga(vaga_suja, lista_padrao,replace_names,  limiar=70):
    vaga_original = vaga_suja  # guardar original caso nada funcione
    vaga_suja  = str(vaga_suja)
    vaga_suja = re.sub('data science', 'data scientist', vaga_suja)
    vaga_suja = vaga_suja.lower().strip()

    # 1. Fuzzy matching
    match, score, _ = process.extractOne(
        vaga_suja,
        lista_padrao,
        scorer=fuzz.token_set_ratio
    )
    if score >= limiar:
        resultado = match
    else:
        # 2. Verificação por inclusão de tokens
        match_fallback = fallback_match(vaga_suja, lista_padrao)
        resultado = match_fallback if match_fallback else vaga_suja

    # 3. Correções finais com replace_names
    for termo, substituto in replace_names.items():
        if termo in resultado:
            resultado = resultado.replace(termo, substituto)

    return resultado


In [13]:
cards['vaga_standard'] =  [padronizar_vaga(vaga_suja, vagas_padrao, replace_names) for vaga_suja in cards['vaga'] ]

In [14]:
cards['vaga_standard'].drop_duplicates().values

array(['cientista de dados', 'analista de dados', 'fbs', 'analista de bi',
       'product analyst', 'oracle technical ai', 'analista atuarial',
       'alin', 'r programming expert contractor', 'estatistico',
       'engenheiro de dados'], dtype=object)

In [15]:
fonte_padrao = ['linkedin', 'indeed', 'vagas', 'catho', 'glassdoor', 'infojobs']
replace_fonte = {'mercado livre' : 'mercado livre'}

In [16]:
cards['fonte_standard'] = [padronizar_vaga(fonte_suja, fonte_padrao, replace_fonte) for fonte_suja in cards['fonte']]

In [17]:
cards['fonte_standard'].value_counts()

fonte_standard
linkedin           222
indeed              42
catho               25
none                 9
glassdoor            6
hiring cafe          5
infojobs             4
trabalha brasil      3
vagas                2
bairesdev site       1
whatsapp             1
team                 1
mercado livre        1
braintrust           1
Name: count, dtype: int64

Using TF-IDF to get word frequencies

In [18]:
# Cleaning desc 



In [19]:
corpus = cards['desc_clean'].dropna()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

palavras = vectorizer.get_feature_names_out()
pesos = X.toarray().sum(axis = 0)

desc_pesos = pd.DataFrame({'palavra': palavras, 'peso_tfidf': pesos}).sort_values('peso_tfidf', ascending=False)

In [20]:
desc_pesos.query("peso_tfidf > 2")

,palavra,peso_tfidf
2803,dados,35.082074
2834,data,19.776989
4234,experiencia,12.376740
2263,conhecimento,9.824690
11121,voce,9.366594
...,...,...
10828,uso,2.019850
1593,carreira,2.015195
5471,implementar,2.012153
4855,gente,2.010829


In [21]:
# Bag of words
vectorizer_bag = CountVectorizer()
X = vectorizer_bag.fit_transform(corpus)

palavras = vectorizer_bag.get_feature_names_out()
contagens = X.toarray().sum(axis = 0)

desc_bag = pd.DataFrame({'palavra': palavras, 'frequência': contagens}).sort_values('frequência', ascending=False)
desc_bag

,palavra,frequência
2803,dados,1644
2834,data,1189
4234,experiencia,526
2263,conhecimento,417
11121,voce,388
...,...,...
5345,hype,1
5344,hyderabad,1
5342,hungry,1
5341,hunger,1


In [22]:
# Using Ollama to extrat job atributes 

OLLAMA_MODEL = 'llama3.2:latest'

def extrair_requisitos(texto_vaga):
    prompt = f"""
A partir da descrição da vaga abaixo, retorne um JSON com os seguintes campos:

- "tecnicas": lista das habilidades técnicas
- "comportamentais": lista das habilidades comportamentais
- "nivel": "junior", "pleno" ou "senior"

Descrição:
\"\"\"{texto_vaga}\"\"\"

Responda apenas com JSON válido.
"""

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False}
    )

    if response.status_code == 200:
        raw = response.json()["response"].strip()
        try:
            return json.loads(raw)
        except json.JSONDecodeError:
            print("⚠️ Erro ao interpretar como JSON. Conteúdo bruto:")
            print(raw)
            return None
    else:
        print("⚠️ Erro na requisição:", response.status_code)
        return None

In [23]:
cards.head(2)

,id,dateLastActivity,desc,due,name,label_names,desc_lang,desc_clean,name_clean,vaga,empresa,fonte,vaga_standard,fonte_standard
0,684c1005b986bba107a1917a,2025-06-13T11:48:36.998Z,about the job job description o time de analy...,2025-07-25T11:44:00.000Z,Cientista de Dados Pleno - Porto - LinkedIn,['Enviado'],portuguese,time analytics insights busca profissional hab...,cientista de dados pleno - porto - linkedin,cientista de dados,porto,linkedin,cientista de dados,linkedin
1,684c138a5be4f9abe78529e7,2025-06-13T12:03:36.307Z,about the job deseja construir um futuro sóli...,2025-07-25T11:44:00.000Z,Analist ade Dados - Cargill - LinkedIn,['Enviado'],portuguese,deseja construir futuro solido sustentavel des...,analist ade dados - cargill - linkedin,analist ade dados,cargill,linkedin,analista de dados,linkedin


In [ ]:
# Suponha que seu DataFrame é `cards` com colunas: id_vaga, desc_clean

resultados = []

for _, row in cards.iterrows():
    requisitos = extrair_requisitos(row['desc_clean'])
    if requisitos:
        nivel = requisitos.get('nivel')
        for tipo, habilidades in [('tecnica', requisitos.get('tecnicas', [])),
                                  ('comportamental', requisitos.get('comportamentais', []))]:
            for habilidade in habilidades:
                resultados.append({
                    'id_vaga': row['id'],
                    'tipo_habilidade': tipo,
                    'habilidade': habilidade
                })
        # adicionar o nível ao DataFrame principal
        cards.loc[cards['id'] == row['id'], 'nivel_estimado'] = nivel


In [126]:
cards['desc_clean'][0]

'time analytics insights busca profissional habilidades analise dados interpretacao metricas geracao insights estrategicos principal responsabilidade transformar grandes volumes dados informacoes objetivas identificando padroes tendencias oportunidades melhoria tambem papel equipe desenvolver manter dashboards relatorios analises preditivas sempre buscando aprimorar eficiencia responsibilities and assignments criacao relatorios dashboards personalizados times estrategicos operacionais area negocio porto bank proporcionando suporte compreensao metricas jornada processo coleta enriquecimento estruturacao bases dados utilizadas construcao relatorios utilizando sistemas disponiveis companhia atraves programacao interpretacao resultados analises geracao insights valiosos comunicando descobertas atraves apresentacoes dashboards relatorios maneira clara compreensivel todos stakeholders acompanhamento continuo relatorios existentes assegurar qualidade informacoes manutencao frequencia adequada

In [151]:
resultados
resultados_df = pd.json_normalize(resultados)
resultados_df.head(2)

,id_vaga,tipo_habilidade,habilidade
0,68513c86c8825584ffc53bd9,tecnica,SQL
1,68513c86c8825584ffc53bd9,tecnica,Excel


In [153]:
resultados_df.to_csv("vaga_requisitos.csv", index = False)

In [18]:
cards.to_csv("cards_clean.csv", index = False)

Get my curriculum in PDf and compare to th ejob positions

In [2]:
cards = pd.read_csv("cards_clean.csv")
cards.head(2)

,id,dateLastActivity,desc,due,name,label_names,desc_lang,desc_clean,name_clean,vaga,empresa,fonte,vaga_standard,fonte_standard
0,684c1005b986bba107a1917a,2025-06-13T11:48:36.998Z,about the job job description o time de analy...,2025-07-25T11:44:00.000Z,Cientista de Dados Pleno - Porto - LinkedIn,['Enviado'],portuguese,time analytics insights busca profissional hab...,cientista de dados pleno - porto - linkedin,cientista de dados,porto,linkedin,cientista de dados,linkedin
1,684c138a5be4f9abe78529e7,2025-06-13T12:03:36.307Z,about the job deseja construir um futuro sóli...,2025-07-25T11:44:00.000Z,Analist ade Dados - Cargill - LinkedIn,['Enviado'],portuguese,deseja construir futuro solido sustentavel des...,analist ade dados - cargill - linkedin,analist ade dados,cargill,linkedin,analista de dados,linkedin


In [3]:
def get_text_pdf(file):
    doc = fitz.open(file)
    texto_en = "" 
    for page in doc:
        texto_en += page.get_text()
    return texto_en

In [4]:
cv_en = get_text_pdf("ESMB170725.pdf")
cv_pt = get_text_pdf("Erika Borelli CV220725.pdf")

In [6]:
OLLAMA_MODEL = 'llama3.2:latest'
BATCH_SIZE = 20

In [7]:
# Salvar raw outputs
def salvar_raw_batch(batch_respostas, nome_arquivo="respostas_raw.jsonl"):
    with open(nome_arquivo, "a", encoding="utf-8") as f:
        for item in batch_respostas:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [8]:

def comparar_vaga_com_cv(descricao_vaga, texto_cv):
    prompt = f"""
Você é um ATS (Applicant Tracking System) avaliando o currículo da candidata em relação à vaga descrita.

Não adicione explicações ou marcações de código. Apenas retorne JSON puro e válido, com os seguintes campos:

- "gaps": uma lista de objetos, cada um contendo:
    - "gap_tag": um identificador padronizado (snake_case) para o gap.
    - "descricao": frase curta explicando por que este é um ponto de melhoria.
    - "tipo": categoria do gap. Pode ser: "tecnico", "idioma", "soft_skill", "experiencia", ou "outro".
    - "origem": se esse requisito estava como "obrigatorio", "desejavel", "implicito" ou "nao_mencionado" na descrição da vaga.
- "score_similaridade": um número de 0 a 100 indicando a compatibilidade geral com a vaga.
- "selecionar_para_proxima_etapa": booleano (true ou false) indicando se a candidata deveria ir para a próxima fase.

Descrição da vaga:
\"\"\"{descricao_vaga}\"\"\" 

Currículo:
\"\"\"{texto_cv}\"\"\"
"""

    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False}
        )
        if response.status_code != 200:
            return {"erro": f"HTTP {response.status_code}", "resposta": response.text}

        raw_text = response.json().get("response", "").strip()
        return {"resposta": raw_text}

    except Exception as e:
        return {"erro": str(e), "resposta": None}


In [ ]:
# respostas_brutas = []

# for i, (_, row) in enumerate(tqdm(cards.iterrows(), total=cards.shape[0])):
#     if not pd.isna(row.get('score_similaridade')):
#         continue

#     cv_file = cv_en if row['desc_lang'] == 'english' else cv_pt

#     try:
#         resultado_llm = comparar_vaga_com_cv(row['desc_clean'], cv_file)

#         respostas_brutas.append({
#             "id_vaga": row['id'],
#             "resposta": resultado_llm
#         })

#     except Exception as e:
#         respostas_brutas.append({
#             "id_vaga": row['id'],
#             "resposta": f"[Erro inesperado] {str(e)}"
#         })


 16%|█▋        | 53/323 [3:25:55<16:37:48, 221.74s/it]

In [9]:
# Lógica em batches
for i in tqdm(range(0, len(cards), BATCH_SIZE)):
    batch = cards.iloc[i:i+BATCH_SIZE]
    respostas_batch = []

    for _, row in batch.iterrows():
        cv = cv_en if row["desc_lang"] == "english" else cv_pt
        resultado = comparar_vaga_com_cv(row["desc_clean"], cv)

        resultado_final = {
            "id_vaga": row["id"],
            "resposta_raw": resultado.get("resposta"),
            "erro": resultado.get("erro", None)
        }

        respostas_batch.append(resultado_final)

    salvar_raw_batch(respostas_batch)

print("✅ Fim da coleta de respostas.")

100%|██████████| 17/17 [15:24:53<00:00, 3264.34s/it]  

✅ Fim da coleta de respostas.


In [10]:
respostas_batch

[{'id_vaga': '6878c8cfa0b87d625b54669e',
  'resposta_raw': '{\n    "gaps": [\n        {\n            "gap_tag": "short-term_experiencia",\n            "descricao": "A vaga menciona a experiência em \'mercado\' como um requisito importante, mas o currículo da candidata não apresenta nenhuma menção a isso.",\n            "tipo": "experiencia",\n            "origem": "nao_mencionado"\n        },\n        {\n            "gap_tag": "tecnico_solid_dados_vizualizacao",\n            "descricao": "O currículo da candidata não menciona a habilidade de criar dashboards utilizando Databricks.",\n            "tipo": "tecnico",\n            "origem": "desejavel"\n        },\n        {\n            "gap_tag": "certificacao_big_data_analytics",\n            "descricao": "A vaga menciona conhecimento em Big Data Analytics, mas o currículo da candidata não apresenta certificação nesse assunto.",\n            "tipo": "certificado",\n            "origem": "nao_mencionado"\n        }\n    ],\n    "score_si

In [28]:
teste = comparar_vaga_com_cv(cards['desc_clean'][1], cv_pt)

In [ ]:
resultados_cv = pd.DataFrame()
avaliacoes = []
dados = cards.head(5).copy()
for i, (_, row) in enumerate(tqdm(dados.iterrows(), total=dados.shape[0])):

    # Evita reprocessar se já tem score
    if not pd.isna(row.get('score_similaridade')):
        continue

    # Seleciona CV adequado
    cv_file = cv_en if row['desc_lang'] == 'english' else cv_pt

    try:
        comparacao = comparar_vaga_com_cv(row['desc_clean'], cv_file)

        if isinstance(comparacao, str):
            print(f"[Erro] Vaga {row['id']}: resposta inválida ou erro detectado.")
            print("Resposta bruta:")
            print(comparacao[:1000])  # Limita o print para não explodir o terminal
            continue
        
        # Se houve erro, ignora
        if "erro" in comparacao:
            print(f"[Erro] Vaga {row['id']}: {comparacao['erro']}")
            continue

        score = comparacao.get('score_similaridade')
        selected = comparacao.get('selecionar_para_proxima_etapa')
        gaps = comparacao.get('gaps', [])

        if isinstance(gaps, list) and gaps:
            gaps_df = pd.json_normalize(gaps)
            gaps_df['id_vaga'] = row['id']
            resultados_cv = pd.concat([resultados_cv, gaps_df], ignore_index=True)

        avaliacoes.append({
            'id': row['id'],
            'score_similaridade': score,
            'selecionar_para_proxima_etapa': selected
        })

        # Salva progresso a cada 10 interações
        if i % 10 == 0:
            resultados_cv.to_csv("gaps_parciais.csv", index=False)
            pd.DataFrame(avaliacoes).to_csv("avaliacoes_parciais.csv", index=False)

    except Exception as e:
        print(f"[Exception] Vaga {row['id']}: {e}")
        continue


 20%|██        | 1/5 [04:34<18:16, 274.25s/it]

[Erro] Vaga 684c1005b986bba107a1917a: resposta inválida ou erro detectado.
Resposta bruta:
[Erro JSON] Expecting value: line 1 column 1 (char 0)
Texto recebido:
```
{
  "gaps": [
    {
      "gap_tag": "tecnico",
      "descricao": "A experiência em IA Generativa e NLP é um pouco descrevida, poderia ser mais detalhada.",
      "tipo": "tecnico",
      "origem": "desejavel"
    },
    {
      "gap_tag": "experiencia",
      "descricao": "A experiência em equipe pode ser melhor documentada.",
      "tipo": "experiencia",
      "origem": "nao_mencionado"
    },
    {
      "gap_tag": "soft_skill",
      "descricao": "A habilidade de comunicar resultados co


 60%|██████    | 3/5 [11:22<07:12, 216.32s/it]

In [40]:
resultados_cv

,gap_tag,descricao,tipo,id_vaga
0,tecnico,Falta de mão-de-obra especializada em Machine ...,tecnico,684c1005b986bba107a1917a
1,outro,A falta de informações sobre a experiência tra...,outro,684c1005b986bba107a1917a


In [41]:

cards.head(2)

,id,dateLastActivity,desc,due,name,label_names,desc_lang,desc_clean,name_clean,vaga,empresa,fonte,vaga_standard,fonte_standard,score_similaridade,selecionar_para_proxima_etapa
0,684c1005b986bba107a1917a,2025-06-13T11:48:36.998Z,about the job job description o time de analy...,2025-07-25T11:44:00.000Z,Cientista de Dados Pleno - Porto - LinkedIn,['Enviado'],portuguese,time analytics insights busca profissional hab...,cientista de dados pleno - porto - linkedin,cientista de dados,porto,linkedin,cientista de dados,linkedin,80.0,True
1,684c138a5be4f9abe78529e7,2025-06-13T12:03:36.307Z,about the job deseja construir um futuro sóli...,2025-07-25T11:44:00.000Z,Analist ade Dados - Cargill - LinkedIn,['Enviado'],portuguese,deseja construir futuro solido sustentavel des...,analist ade dados - cargill - linkedin,analist ade dados,cargill,linkedin,analista de dados,linkedin,NaN,NaN


In [39]:

comparacao = comparar_vaga_com_cv(cards['desc_clean'][0], cv_pt)

In [41]:
print(comparacao)

{
  "pontos_fortes": [
    "Análise de dados estatísticos com Python e R",
    "Experiência em desenvolvimento de dashboards interativos",
    "Conhecimento em machine learning com Python e Power BI",
    "Facilidade em comunicar resultados complexos a públicos diversos"
  ],
  "gaps": [
    {
      "gap_tag": "experiência_companhia_bank",
      "descricao": "Falta de experiência em bancos ou setores financeiros",
      "tipo": "outro"
    },
    {
      "gap_tag": "conhecimento_productos_financeiros",
      "descricao": "Falta de conhecimento em produtos financeiros e instrumentos de crédito",
      "tipo": "outro"
    }
  ],
  "score_similaridade": 80,
  "selecionar_para_proxima_etapa": true
}
